In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
from dark_emulator_public import dark_emulator
import os, sys, time
import matplotlib.pyplot as plt
from collections import OrderedDict as od
from scipy.interpolate import InterpolatedUnivariateSpline as ius
from scipy.interpolate import interp2d, interp1d
from scipy.integrate import simps
from tqdm import tqdm
import hsc3x2pt

using dark_emulator at  /lustre/work/sunao.sugiyama/package/dark_emulator_public/dark_emulator/__init__.py


## Preparation for Fisher
Computation of $C(l)$ takes ~20 sec for each cosmology.
So this notebook pre-compute $C(l)$ for cosmologies needed for Fisher analysis.

Model parameters of 3x2pt analysis with single(double) source redshift bin are
- comsology: 5
- galaxy bias: 1x3
- magnification bias: 1x3
- photo-z: 1x1(2)
- multiplicative bais: 1x1(2)

13(15) parameters in total.

$C(l)$s depend on cosmological parameters, galaxy bias and magnification bias, for which we need to compute $C(l)$.

**Note** : Fisher analysis is doable even if the computational time for one model reaches ~20 sec. However, in the real analysis, ~20 sec is too much to run sampling code. 
I just want to know how people in cosmic shear, like Hikage-san and Hamana-san, or people in 3x2pt, like DES or KiDS, implement code of $C(l)$.?

In [5]:
power_b1 = hsc3x2pt.power_b1_class()

initialize cosmo_class
Initialize pklin emulator
initialize propagator emulator
Initialize sigma_d emulator
initialize cross-correlation emulator
initialize auto-correlation emulator
Initialize sigmaM emulator
initialize xinl emulator


### List of models to compute $C(l)$ to compute **single** source bin anlaysis
0. fiducial = Planck cosmology + fiducial galaxy bias, magnification bais + 0 photo-z and multiplicative bias
1. $\omega_b h^2$ +
2. $\omega_c h^2$ +
3. $\Omega_{\rm de}$ +
4. $n_{\rm s}$ +
5. $\ln 10^{10}A_{\rm s}$ +
6. $b_{\rm 1,1}$ + 
7. $b_{\rm 1,2}$ + 
8. $b_{\rm 1,3}$ + 
9. $\alpha_{\rm mag,1}$ + 
10. $\alpha_{\rm mag,2}$ + 
11. $\alpha_{\rm mag,3}$ + 
12. $\Delta z_{\rm ph}$ +
13. $\Delta m$ + = easy. Just multiply $1+\Delta m$ to fiducial.

In [18]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s12', 'sourcePzs_Y1/MLZs12.txt', 0.0, 0.2, '8.74 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
pk2cl.set_cosmology_from_dict()
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':140, 'xi':140}) # HSCY1

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache('single_source_Cl/fiducial', overwrite=True)

:13.081504821777344 sec
saving l to single_source_Cl/fiducial/l.txt
saving lowz,lowz to single_source_Cl/fiducial/lowz,lowz.txt
saving cmass1,cmass1 to single_source_Cl/fiducial/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_Cl/fiducial/cmass2,cmass2.txt
saving lowz,s12 to single_source_Cl/fiducial/lowz,s12.txt
saving cmass1,s12 to single_source_Cl/fiducial/cmass1,s12.txt
saving cmass2,s12 to single_source_Cl/fiducial/cmass2,s12.txt
saving s12,s12 to single_source_Cl/fiducial/s12,s12.txt
saving cosmo_dict to single_source_Cl/fiducial/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
CPU times: user 13.4 s, sys: 14.6 ms, total: 13.4 s
Wall time: 13.4 s


In [17]:
g_s12.z_source_eff

array(1.14660218)

In [9]:
cosmo_dict = pk2cl.get_cosmo_dict()

In [10]:
cosmo_dict

{'omega_b': 0.02225,
 'omega_c': 0.1198,
 'Omega_de': 0.6844,
 'ln10p10As': 3.094,
 'n_s': 0.9645,
 'w_de': -1.0,
 'Omega_de0': 0.6844,
 'Omega_K0': 0.0,
 'w0': -1.0,
 'wa': 0.0,
 'h': 0.6724008777737618}

### List of models to compute $C(l)$ to compute **double** source bins anlaysis

0. fiducial = Planck cosmology + fiducial galaxy bias, magnification bais + 0 photo-z and multiplicative bias
1. $\omega_b h^2$ +
2. $\omega_c h^2$ +
3. $\Omega_{\rm de}$ +
4. $n_{\rm s}$ +
5. $\ln 10^{10}A_{\rm s}$ +
6. $b_{\rm 1,1}$ + 
7. $b_{\rm 1,2}$ + 
8. $b_{\rm 1,3}$ + 
9. $\alpha_{\rm mag,1}$ + 
10. $\alpha_{\rm mag,2}$ + 
11. $\alpha_{\rm mag,3}$ + 
12. $\Delta z_{\rm ph,1}$ +
12. $\Delta z_{\rm ph,2}$ +
13. $\Delta m_1$ + = easy. Just multiply $1+\Delta m_1$ to fiducial.
13. $\Delta m_2$ + = easy. Just multiply $1+\Delta m_2$ to fiducial.

In [ ]:
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.251, 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.484, 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.617, 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s1 = hsc3x2pt.galaxy_sample_source_class(['s1' , 1.0, 0.2, '4.37 arcmin^-2'])
g_s2 = hsc3x2pt.galaxy_sample_source_class(['s2' , 1.4, 0.2, '4.37 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s1)
pk2cl.set_galaxy_sample(g_s2)
pk2cl.set_cosmology_from_dict()
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':140, 'xi':140}) # HSCY1